# Phase Estimation

The **"Phase Estimation"** quantum kata is a series of exercises designed
to teach you the basics of using phase estimation algorithms.

It covers the following topics:
* quantum phase estimation,
* iterative phase estimation,
* preparing necessary inputs to phase estimation routines and applying them.

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

Within each section, tasks are given in approximate order of increasing difficulty; 
harder ones are marked with asterisks.

## Part I. Quantum Phase Estimation (QPE)

### Task 1.1. Inputs to QPE: eigenstates of Z/S/T gates.

**Inputs:** 

  1. A qubit in the $|0\rangle$ state.

  2. An integer `state` indicating which eigenstate to prepare.

**Goal:** 

Prepare one of the eigenstates of Z gate (which are the same as eigenstates of S or T gates): 
eigenstate $|0\rangle$ if `state = 0`, or eigenstate $|1\rangle$ if `state = 1`.

In [1]:
%kata T11_Eigenstates_ZST 

operation Eigenstates_ZST (q : Qubit, state : Int) : Unit is Adj {
    // [Pauli] Z (computational) basis single-qubit eigenstates are 0 and 1.
    if (state == 1) { X(q); }
}

Success!

### Task 1.2. Inputs to QPE: powers of Z/S/T gates.

**Inputs:** 

  1. A single-qubit unitary U.

  2. A positive integer `power`.

**Output:** 

A single-qubit unitary equal to U raised to the given power.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Remember that you can define auxiliary operations. To do that, you'll need to create an extra code cell for each new operation and execute it before returning to this cell. 
</details>

In [12]:
%kata T12_UnitaryPower 

function UnitaryPower (U : (Qubit => Unit is Adj + Ctl), power : Int) : (Qubit => Unit is Adj + Ctl) {
    //
    return OperationPowCA<Qubit>(U, power);
}

Success!

### Task 1.3. Validate inputs to QPE.

<span style="color:red"><b>This task is temporarily not available in Notebook format; please use Q# project version of the PhaseEstimation kata to complete it.</b></span>

**Inputs:**

  1. A single-qubit unitary U.

  2. A single-qubit state $|\psi\rangle$ represented by a unitary P such that $|\psi\rangle = P|0\rangle$
(i.e., applying the unitary P to state $|0\rangle$ prepares state $|\psi\rangle$).

**Goal:** 

Assert that the given state is an eigenstate of the given unitary, 
i.e., do nothing if it is, and throw an exception if it is not.

In [ ]:
operation AssertIsEigenstate (U : (Qubit => Unit), P : (Qubit => Unit is Adj)) : Unit {
    use q = Qubit();
    // Prepare the state |ψ⟩
    P(q);
    // Apply the given unitary
    U(q);
    // If the given state is an eigenstate, the current state of the qubit should be |ψ⟩ (up to a global phase).
    // So un-preparing it should bring the state back to |0⟩
    Adjoint P(q);
    AssertQubit(Zero, q);
}

In [ ]:
%simulate AssertIsEigenstate 

***
In general, a unitary operator has more than 2 eigenvectors.

Two-qubit operators have 4 eigenvectors. In the Fourier basis, each has a phase.

Each eigenvector is a superposition of single-qubit computational basis states,
acted on by a phase shift matrix (Z, S, T) a variable number of times.

Therefore, in the Fourier basis, each component is an eigenstate of powers of a phase shift operator.

***

### Task 1.4. QPE for single-qubit unitaries.

**Inputs:** 

  1. A single-qubit unitary U.

  2. A single-qubit state $|\psi\rangle$ represented by a unitary P such that $|\psi\rangle = P|0\rangle$
(i.e., applying the unitary P to state $|0\rangle$ prepares state $|\psi\rangle$).

  3. An integer `n`.

**Output:**

The phase of the eigenvalue that corresponds to the eigenstate $|\psi\rangle$, with `n` bits of precision.
The phase should be between 0.0 and 1.0.

In [11]:
operation PowerOracle (U : (Qubit => Unit is Adj + Ctl), power : Int, target : Qubit[]) : Unit is Adj + Ctl {
    //
    RepeatCA(U(_), power, target[0]);
}

PowerOracle

In [16]:
%kata T14_QPE 

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Oracles;
open Microsoft.Quantum.Characterization;

operation QPE (U : (Qubit => Unit is Adj + Ctl), P : (Qubit => Unit is Adj), n : Int) : Double {
    // Construct a phase estimation oracle from the unitary
    let oracle = DiscreteOracle(PowerOracle(U, _, _));
    // Allocate qubits to hold the eigenstate of U and the phase in a big endian register 
    use (eigenstate, phaseRegister) = (Qubit[1], Qubit[n]);
    let phaseRegisterBE = BigEndian(phaseRegister);
    // Prepare the eigenstate of U
    P(eigenstate[0]);
    // Call library
    QuantumPhaseEstimation(oracle, eigenstate, phaseRegisterBE);
    // Read out the phase
    let phase = IntAsDouble(MeasureInteger(BigEndianAsLittleEndian(phaseRegisterBE))) / IntAsDouble(1 <<< n);

    ResetAll(eigenstate);
    return phase;
}

Success!

In [18]:
%kata T14_QPE 

operation QPE (U : (Qubit => Unit is Adj + Ctl), P : (Qubit => Unit is Adj), n : Int) : Double {
    use (eigenstate, phaseRegister) = (Qubit(), Qubit[n]);
    // Prepare the eigenstate of U
    P(eigenstate);
    // Put phaseRegister into a uniform superposition of all computational basis states.
    ApplyToEachA(H, phaseRegister);
    // Apply controlled U gates so that |k⟩ gets transformed into exp(2πikθ)|k⟩.
    // Here |k⟩ is encoded in little-endian format in phaseRegister.
    for i in 0 .. n - 1 {
        let powU = UnitaryPower(U, 1 <<< i); // powU = U^(2ⁱ)
        Controlled powU([phaseRegister[i]], eigenstate);
    }
    // Apply an inverse QFT to phaseRegister.
    // We use QFTLE because phaseRegister is little-endian encoded.
    // This leaves phaseRegister in a superposition of integers which are close to θ*2^n.
    // See the linked Wikipedia article on the details of this superposition.
    Adjoint QFTLE(LittleEndian(phaseRegister));
    // Measure out an integer that is close to θ*2^n.
    let phaseRegisterMeasurement = MeasureInteger(LittleEndian(phaseRegister));
    // Clean up eigenstate qubits
    Reset(eigenstate);
    // Scale our measurement down to [0, 1) and return our estimate.
    let phaseEstimate = IntAsDouble(phaseRegisterMeasurement) / IntAsDouble(1 <<< n);
    return phaseEstimate;
}

Success!

### Task 1.5. Test your QPE implementation.

**Goal:**
Use your QPE implementation from task 1.4 to run quantum phase estimation 
on several simple unitaries and their eigenstates.
This task is not covered by a test and allows you to experiment with running the algorithm.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_QPE` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_QPE`).

In [ ]:
operation Run_QPE () : Unit {
    // ...
}

In [ ]:
%simulate Run_QPE

## Part II. Iterative Phase Estimation

Unlike quantum phase estimation, which is a single algorithm, 
iterative phase estimation is a whole class of algorithms based on the same idea:
treating phase estimation as a classical algorithm which learns the phase via a sequence of measurements
(the measurement performed on each iteration can depend on the outcomes of previous iterations).

A typical circuit for one iteration has the following structure:

![Iterative Phase Estimation Circuit Diagram](./img/IPE_Circuit.PNG)

($\psi$ is the procedure to prepare the eigenstate $|\psi\rangle$, R is a rotation gate, and M is a power of the unitary U;
both depend on the current information about the phase).

The result of the measurement performed on the top qubit defines the next iteration.

### Task 2.1. Single-bit phase estimation.

**Inputs:** 

  1. A single-qubit unitary U that is guaranteed to have an eigenvalue $+1$ or $-1$ 
(with eigenphases $0.0$ or $0.5$, respectively).

  2. A single-qubit state $|\psi\rangle$ represented by a unitary P such that $|\psi\rangle = P|0\rangle$
(i.e., applying the unitary P to state $|0\rangle$ prepares state $|\psi\rangle$).

**Output:** 

The eigenvalue which corresponds to the eigenstate $|\psi\rangle$ ($+1$ or $-1$).

You are allowed to allocate exactly two qubits and call `Controlled U` exactly once.

> It is possible to use the QPE implementation from task 1.4 to solve this task,
  but we suggest you implement the circuit by hand for the sake of learning.

In [ ]:
%kata T21_SingleBitPE 

operation SingleBitPE (U : (Qubit => Unit is Adj + Ctl), P : (Qubit => Unit is Adj)) : Int {
    // ...
}

### Task 2.2. Two bit phase estimation.

**Inputs:**

  1. A single-qubit unitary U that is guaranteed to have an eigenvalue $+1$, $i$, $-1$ or $-i$
(with eigenphases $0.0$, $0.25$, $0.5$ or $0.75$, respectively).

  2. A single-qubit state $|\psi\rangle$ represented by a unitary P such that $|\psi\rangle = P|0\rangle$
(i.e., applying the unitary P to state $|0\rangle$ prepares state $|\psi\rangle$).

**Output:**

The eigenphase which corresponds to the eigenstate $|\psi\rangle$ ($0.0$, $0.25$, $0.5$ or $0.75$).
The returned value has to be accurate within the absolute error of 0.001.

You are allowed to allocate exactly two qubits and call `Controlled U` multiple times.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Start by applying the same circuit as in task 2.1.  
  What are the possible outcomes for each eigenvalue?  
  What eigenvalues you can and can not distinguish using this circuit?
</details>

<br/>
<details>
  <summary><b>Need another hint? Click here</b></summary>
  What eigenvalues you can and can not distinguish using this circuit?
  What circuit you can apply to distinguish them?
</details>

In [ ]:
%kata T22_TwoBitPE 

operation TwoBitPE (U : (Qubit => Unit is Adj + Ctl), P : (Qubit => Unit is Adj)) : Double {
    // ...
    return -1.0;
}

To be continued...